# elmo_sentence_level_embedding

Example use https://tfhub.dev/google/elmo/2

Text embeddings can be performed at individual word level present inside the sentence or can be performed at the entire sentence level. 

**This paper deals with sentence level**.

```
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
embeddings = elmo(
["the cat is on the mat", "dogs are in the fog"],
signature="default",
as_dict=True)["elmo"]
```

## 0. Dead work
1. Download the data from [Kaggle's movie review sentiment analysis](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews)
2. Unzip the downloaded files,store train.tsv and test.tsv to data file.

## 1. import package

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import os
from tqdm import tqdm
import csv

## 2. Data

### 2.1 Preprocessing data

In [2]:
class Data:
    def __init__(self, params):
        self._train_file = os.path.abspath(params['TRAIN_FILE'])
        self._test_file = os.path.abspath(params['TEST_FILE'])
        self._train_val_split = params['TRAIN_VAL_RATIO']

        self._fetch_data()

    def _fetch_data(self):
        X_data, y_data = self._read_train_data()
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(X_data, y_data,
                                                                        train_size=self._train_val_split)

        self.X_test, self.X_test_id = self._read_test_data()

    def _read_train_data(self):
        pd_data = pd.read_csv(self._train_file, sep='\t')
        X_data, y_data = pd_data['Phrase'], pd_data['Sentiment']
        return X_data, y_data

    def _read_test_data(self):
        pd_data = pd.read_csv(self._test_file, sep='\t')
        X_data, X_data_id = pd_data['Phrase'], pd_data['PhraseId']
        return X_data, X_data_id

    def get_train_data_length(self):
        return len(self.y_train)

    def get_val_data_length(self):
        return len(self.y_val)

    def get_test_data_length(self):
        return len(self.X_test_id)

### 2.2 Dataset

In [3]:
class Dataset:
    def __init__(self, params):
        self._n_class = params['N_CLASS']
        self._batch_size = params['BATCH_SIZE']

        self._create_iterator()

    def _create_iterator(self):
        self._pl_phrase_id = tf.placeholder(tf.int32, (None), name='pl_phrase_id')
        self._pl_phrase_text = tf.placeholder(tf.string, (None), name='pl_phrase_text')
        self._pl_sentiment = tf.placeholder(tf.int32, (None), name='pl_sentiment')

        dataset = tf.data.Dataset.from_tensor_slices((self._pl_phrase_id, self._pl_phrase_text,
                                                      self._pl_sentiment))
        dataset = dataset.batch(self._batch_size)

        iterator = tf.data.Iterator.from_structure(dataset.output_types)
        self._iterator_initializer = iterator.make_initializer(dataset, name='initializer')
        self.text_id, self.text_data, self.labels = iterator.get_next()
        self.text_id = tf.identity(self.text_id, name='text_id')

    def initialize_iterator(self, sess, phrase_text, sentiment):
        phrase_id = np.zeros((len(phrase_text)), dtype=np.int32)
        feed_dict = {
            self._pl_phrase_id: phrase_id,
            self._pl_phrase_text: phrase_text,
            self._pl_sentiment: sentiment
        }
        sess.run(self._iterator_initializer, feed_dict=feed_dict)

    def initialize_test_iterator_for_saved_model_graph(self, sess, phrase_id, phrase_text):
        pl_phrase_id = sess.graph.get_tensor_by_name('pl_phrase_id:0')
        pl_phrase_text = sess.graph.get_tensor_by_name('pl_phrase_text:0')
        pl_sentiment = sess.graph.get_tensor_by_name('pl_sentiment:0')
        initializer = sess.graph.get_operation_by_name('initializer')

        sentiment = np.zeros((len(phrase_text)), dtype=np.float32)
        feed_dict = {
            pl_phrase_id: phrase_id,
            pl_phrase_text: phrase_text,
            pl_sentiment: sentiment
        }
        sess.run(initializer, feed_dict=feed_dict)

## 3. Model

In [4]:
class Model:
    def __init__(self, params):
        self._n_class = params['N_CLASS']
        self._prepare_graph(params)

    def _prepare_graph(self, params):
        tf.reset_default_graph()

        self.dataset = Dataset(params)

        self.lr = tf.placeholder(tf.float32, shape=())
        one_hot_y = tf.one_hot(self.dataset.labels, depth=self._n_class)
        logits = self._prepare_model(self.dataset.text_data)

        cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=one_hot_y)
        self.loss = tf.reduce_sum(cross_entropy)

        self.optimizer = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(self.loss)

        self.predictions = tf.argmax(logits, axis=1, output_type=tf.int32, name='predictions')
        self.accuracy = tf.reduce_sum(tf.cast(tf.equal(self.predictions, self.dataset.labels), tf.float32))

    def _prepare_model(self, text):
        module = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        embeddings = module(dict(text=text)) #shape:[batch_szie, 1024]
        #print(embeddings)
        logits = tf.layers.dense(embeddings, units=self._n_class)
        return logits

## 4. Train

In [5]:
class Train:
    def __init__(self, params):
        self._epochs = params['EPOCHS']
        self._batch_size = params['BATCH_SIZE']
        self._lr = params['LEARNING_RATE']
        self._n_class = params['N_CLASS']

        self.data = Data(params)
        self.model = Model(params)

        self._save_path = os.path.abspath('./Model_OUTPUT(sentence_level_elmo)')

    def train(self):
        shutil.rmtree(self._save_path, ignore_errors=True)
        os.mkdir(self._save_path)

        with tf.Session() as sess:
            sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
            current_lr = self._lr

            for epoch_no in range(self._epochs):
                train_loss, train_accuracy = 0, 0
                val_loss, val_accuracy = 0, 0

                print('\nEpoch: {}, lr: {:.6f}'.format(epoch_no + 1, current_lr))
                self.model.dataset.initialize_iterator(sess, self.data.X_train, self.data.y_train)
                try:
                    with tqdm(total=self.data.get_train_data_length()) as pbar:
                        while True:
                            _, l, a = sess.run([self.model.optimizer, self.model.loss, self.model.accuracy],
                                               feed_dict={self.model.lr: current_lr})
                            train_loss += l
                            train_accuracy += a
                            pbar.update(self._batch_size)
                except tf.errors.OutOfRangeError:
                    pass

                self.model.dataset.initialize_iterator(sess, self.data.X_val, self.data.y_val)
                try:
                    with tqdm(total=self.data.get_val_data_length()) as pbar:
                        while True:
                            l, a = sess.run([self.model.loss, self.model.accuracy])
                            val_loss += l
                            val_accuracy += a
                            pbar.update(self._batch_size)
                except tf.errors.OutOfRangeError:
                    pass

                train_accuracy /= self.data.get_train_data_length()
                train_loss /= self.data.get_train_data_length()
                val_accuracy /= self.data.get_val_data_length()
                val_loss /= self.data.get_val_data_length()

                print('Train accuracy: {:.4f}, loss: {:.4f}'.format(train_accuracy, train_loss))
                print('Validation accuracy: {:.4f}, loss: {:.4f}'.format(val_accuracy, val_loss))
                self._save_model(sess, epoch_no)

    def test(self):
        test_graph = tf.Graph()
        with test_graph.as_default():
            with tf.Session(graph=test_graph) as sess, \
                    open(os.path.join(self._save_path, 'results.csv'), 'w') as fid:
                sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
                saved_model_path = os.path.join(self._save_path, str(self._epochs - 1))
                tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], saved_model_path)
                self.model.dataset.initialize_test_iterator_for_saved_model_graph(sess, self.data.X_test_id,
                                                                    self.data.X_test)

                csv_fid = csv.writer(fid)
                csv_fid.writerow(['PhraseId', 'Sentiment'])

                predictions_op = test_graph.get_tensor_by_name('predictions:0')
                text_id_op = test_graph.get_tensor_by_name('text_id:0')
                try:
                    with tqdm(total=self.data.get_test_data_length()) as pbar:
                        while True:
                            predictions, phrase_id = sess.run([predictions_op, text_id_op])
                            predictions = predictions.tolist()
                            phrase_id = phrase_id.tolist()

                            for pred, p in zip(predictions, phrase_id):
                                csv_fid.writerow([p, pred])
                            pbar.update(self._batch_size)
                except tf.errors.OutOfRangeError:
                    pass

    def _save_model(self, sess, epoch_no):
        inputs = {
            'pl_phrase_id': sess.graph.get_tensor_by_name('pl_phrase_id:0'),
            'pl_phrase_text': sess.graph.get_tensor_by_name('pl_phrase_text:0'),
            'pl_sentiment': sess.graph.get_tensor_by_name('pl_sentiment:0')
        }
        outputs = {'accuracy': self.model.accuracy}

        export_dir = os.path.join(self._save_path, str(epoch_no))
        tf.saved_model.simple_save(sess, export_dir, inputs, outputs)

## Experiment

In [ ]:
params = {
    'EPOCHS': 6,
    'BATCH_SIZE': 64,
    'LEARNING_RATE': 0.0001,

    'TRAIN_FILE': './Data/train.tsv',
    'TEST_FILE': './Data/test.tsv',
    'TRAIN_VAL_RATIO': 0.95,

    'N_CLASS': 5
}

t = Train(params)
t.train()
t.test()

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


/home/b418/anaconda3/envs/yuanxiao/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore

Epoch: 1, lr: 0.000100


 10%|█         | 15488/148257 [1:05:25<9:30:25,  3.88it/s] 